In [1]:
import pandas as pd
import re 
import spacy 
import os 
import numpy as np
import altair as alt
import textdistance
from sklearn.preprocessing import MultiLabelBinarizer
alt.data_transformers.enable('json')
# pd.set_option('display.max_rows', 500)
# pd.set_option('display.max_columns', 500)
# pd.set_option('display.width', 1000)

DataTransformerRegistry.enable('json')

In [2]:
nlp = spacy.load("en_core_web_sm")

In [3]:
df_work_exp = pd.read_csv("../../../Glentel Inc/HR Analytics - Documents/Capstone Data/ubc_mds_team_share/06_clean_data/work_experience.csv")
df_work_exp.columns = df_work_exp.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
df_work_exp = df_work_exp.drop(['work1_company', 'work2_company','work3_company', 'work4_company', 
                                'work5_company', 'work6_company', 'work7_company'], axis=1)


In [4]:
df_work_exp = df_work_exp.fillna(0)
for i in range(1, 7):
    df_work_exp['work'+str(i)+'_length'] = np.abs(df_work_exp['work'+str(i)+'_length'])

In [17]:
df_industry = pd.read_csv("../../../Glentel Inc/HR Analytics - Documents/Capstone Data/ubc_mds_team_share/03_feature_extraction/output/job_category_features_2020_06_04_V1.csv")
df_industry.columns = df_industry.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
df_industry = df_industry.drop(['work1_company', 'work2_company', 'work3_company',
       'work4_company', 'work5_company', 'work6_company', 'work7_company'], axis=1)


In [18]:
df_industry.columns

Index(['employee_code', 'industry_1', 'industry_2', 'industry_3', 'industry_4',
       'industry_5', 'industry_6', 'industry_7'],
      dtype='object')

In [19]:
df_industry = pd.merge(df_industry, df_work_exp, on='employee_code')

In [ ]:
d

,employee_code,industry_1,industry_2,industry_3,industry_4,industry_5,industry_6,industry_7,work1_length,work2_length,work3_length,work4_length,work5_length,work6_length
0,BDP,Telecomunications,Telecomunications,NaN,NaN,NaN,NaN,NaN,44.0,63.0,36.0,NaN,NaN,NaN
1,F6Q,Telecomunications,Telecomunications,NaN,NaN,NaN,NaN,NaN,NaN,22.0,NaN,NaN,NaN,NaN
2,MRN,Other,Other,NaN,NaN,NaN,NaN,NaN,24.0,24.0,NaN,NaN,NaN,NaN
3,MXO,Sport_Travel_Enterntain_Hotel,Sport_Travel_Enterntain_Hotel,Sport_Travel_Enterntain_Hotel,Sport_Travel_Enterntain_Hotel,NaN,NaN,NaN,54.0,21.0,7.0,26.0,NaN,NaN
4,N0Y,Sport_Travel_Enterntain_Hotel,Consumer electronics,unknown,unknown,NaN,NaN,NaN,4.0,14.0,19.0,8.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284,MUD,unknown,Other,Telecomunications,unknown,NaN,NaN,NaN,8.0,3.0,5.0,6.0,NaN,NaN
285,L46,unknown,Telecomunications,unknown,NaN,NaN,NaN,NaN,7.0,23.0,32.0,NaN,NaN,NaN
286,N87,Food-Convenience-Pharmacy,unknown,Food-Convenience-Pharmacy,NaN,NaN,NaN,NaN,22.0,37.0,15.0,NaN,NaN,NaN
287,ND9,Food-Convenience-Pharmacy,unknown,Food-Convenience-Pharmacy,Consumer electronics,NaN,NaN,NaN,32.0,41.0,7.0,16.0,NaN,NaN


In [10]:
food_serv_ind = np.zeros(df_industry.shape[0])
apparel_ind = np.zeros(df_industry.shape[0])
food_conv_phar_ind = np.zeros(df_industry.shape[0])
automotive_ind = np.zeros(df_industry.shape[0])
blue_collar_ind = np.zeros(df_industry.shape[0])
consum_elect_ind = np.zeros(df_industry.shape[0])

for i in range(0, df_industry.shape[0]):
    for j in range(1,7):
        if df_industry['food_serving'+str(j)][i] == 1:
            food_serv_ind[i] += df_industry['work'+str(j)+'_length'][i]
        if df_industry['cf_wear'+str(j)][i] == 1:
            apparel_ind[i] += df_industry['work'+str(j)+'_length'][i]
        if df_industry['food_conv_phar'+str(j)][i] == 1:
            food_conv_phar_ind[i] += df_industry['work'+str(j)+'_length'][i]
        if df_industry['car_dealer'+str(j)][i] == 1:
            automotive_ind[i] += df_industry['work'+str(j)+'_length'][i]
        if df_industry['c_electronics'+str(j)][i] == 1:
            consum_elect_ind[i] += df_industry['work'+str(j)+'_length'][i]
        if df_industry['blue_collar'+str(j)][i] == 1:
            blue_collar_ind[i] += df_industry['work'+str(j)+'_length'][i]

In [11]:
df_industry_exp = df_industry[['employee_code']]

In [21]:
df_industry_exp['food_service_industry_exp'] = food_serv_ind.tolist()
df_industry_exp['apparel_industry_exp'] = apparel_ind.tolist()
df_industry_exp['supercenter_convenience_industry_exp'] = food_conv_phar_ind.tolist()
df_industry_exp['automotive_sales_industry_exp'] = automotive_ind.tolist()
df_industry_exp['blue_collar_industry_exp'] = blue_collar_ind.tolist()
df_industry_exp['consumer_electronics'] = consum_elect_ind.tolist()

C:\Users\NIY\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\NIY\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\NIY\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the d

In [22]:
df_industry_exp

,employee_code,food_service_industry_exp,apparel_industry_exp,consumer_goods_industry_exp,automotive_goods_industry_exp,blue_collar_industry_exp,consumer_electronics,supercenter_convenience_industry_exp,automotive_sales_industry_exp
0,BDP,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,F6Q,0.0,0.0,0.0,0.0,0.0,22.0,0.0,0.0
2,MRN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,MXO,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,N0Y,0.0,0.0,0.0,14.0,0.0,14.0,0.0,14.0
...,...,...,...,...,...,...,...,...,...
284,MUD,0.0,0.0,0.0,3.0,0.0,5.0,0.0,3.0
285,L46,0.0,23.0,0.0,0.0,0.0,0.0,0.0,0.0
286,N87,0.0,0.0,22.0,0.0,0.0,0.0,22.0,0.0
287,ND9,0.0,0.0,39.0,23.0,0.0,23.0,39.0,23.0


In [24]:
df_industry_exp.to_csv('../../../Glentel Inc/HR Analytics - Documents/Capstone Data/ubc_mds_team_share/06_clean_data/industry_exp.csv')